In [1]:
%env CUDA_VISIBLE_DEVICES=0

env: CUDA_VISIBLE_DEVICES=0


In [ ]:
import torch
import torch.nn as nn
import pandas as pd
#import math
import matplotlib.pyplot as plt
from ccvae_model import *
from process_data import *
from m2_model import *

import numpy as np
#from torchvision.utils import make_grid, save_image
import torch.distributions as dist
from torch.utils.data import DataLoader

from nltk.tokenize import word_tokenize

from sklearn.model_selection import train_test_split
import torch.optim as optim
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

[nltk_data] Downloading package punkt to /home/vcourrier/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/vcourrier/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/vcourrier/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


device(type='cpu')

In [3]:
df = pd.read_csv("data/IMDB_Dataset.csv")
print(df.head(), "\n")

                                              review sentiment
0  One of the other reviewers has mentioned that ...  positive
1  A wonderful little production. <br /><br />The...  positive
2  I thought this was a wonderful way to spend ti...  positive
3  Basically there's a family where a little boy ...  negative
4  Petter Mattei's "Love in the Time of Money" is...  positive 



In [ ]:
vocab_size = 10000
seq_len = 100

final_df, word2value = process_IMDB("data/IMDB_Dataset.csv", vocab_size = vocab_size, max_pad=seq_len)

####### Train-test split ########

# Split the data into training and testing sets (80% training, 20% testing)
train_df, test_df = train_test_split(final_df, test_size=0.15, random_state=42)

print("Training set shape:", train_df.shape)
print("Testing set shape:", test_df.shape)

Training set shape: (42500, 2)
Testing set shape: (7500, 2)


In [6]:
# Define batch size and number of epochs
batch_size = 200

# Create custom datasets
train_dataset = CustomDataset(train_df)
test_dataset = CustomDataset(test_df)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

data_train = iter(train_loader)

In [7]:
input_size = 1 
hidden_dim = 256
z_dim = 28
num_classes = 1
prior_fn = torch.tensor([[0.5]])
n_epochs = 10

In [8]:
num_batch = int(len(train_dataset)/batch_size)
num_batch

212

In [12]:
def init_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight)
        if m.bias is not None:
            nn.init.zeros_(m.bias)
    elif isinstance(m, nn.LSTM):
        for name, param in m.named_parameters():
            if 'weight' in name:
                nn.init.xavier_uniform_(param)
            elif 'bias' in name:
                nn.init.zeros_(param)

In [ ]:
import gc

# Training
cc_vae = CCVAE(input_size, hidden_dim, vocab_size, z_dim, num_classes, use_cuda, prior_fn, device).to(device=device)

cc_vae.apply(init_weights)

optim = torch.optim.Adam(params=cc_vae.parameters(), lr=0.0001)

for epoch in range(n_epochs):
    cc_vae.train()
    
    epoch_losses_sup = 0.0
    for batch_idx, (_, xs, ys) in enumerate(train_loader):
        xs = xs.view(xs.shape[0], seq_len, input_size).to(device)
        ys = ys.to(device)
        loss = cc_vae.sup(xs, ys)
        
        optim.zero_grad() 
        loss.backward()
        optim.step()

        epoch_losses_sup += loss.detach().item()
        #print(f"Batch {batch_idx} Loss: {loss.detach().item():.3f}")
        gc.collect()
    
    avg_loss = epoch_losses_sup / len(train_loader)
    print(f"[Epoch {epoch+1:03d}] Sup Loss: {avg_loss:.3f}")


Epoch: 0
  Batch: 0
Batch 0 Loss: 20.755
  Batch: 1
Batch 1 Loss: 19.612
  Batch: 2
Batch 2 Loss: 19.539
  Batch: 3
Batch 3 Loss: 19.414
  Batch: 4
Batch 4 Loss: 19.211
  Batch: 5
Batch 5 Loss: 19.195
  Batch: 6
Batch 6 Loss: 18.643
  Batch: 7
Batch 7 Loss: 19.083
  Batch: 8
Batch 8 Loss: 19.013
  Batch: 9
Batch 9 Loss: 17.426
  Batch: 10
Batch 10 Loss: 17.308
  Batch: 11
Batch 11 Loss: 16.716
  Batch: 12
Batch 12 Loss: 17.119
  Batch: 13
Batch 13 Loss: 18.053
  Batch: 14
Batch 14 Loss: 16.453
  Batch: 15
Batch 15 Loss: 15.370
  Batch: 16
Batch 16 Loss: 15.892
  Batch: 17
Batch 17 Loss: 14.507
  Batch: 18
Batch 18 Loss: 15.216
  Batch: 19
Batch 19 Loss: 15.513
  Batch: 20
Batch 20 Loss: 15.383
  Batch: 21
Batch 21 Loss: 15.155
  Batch: 22
Batch 22 Loss: 14.869
  Batch: 23
Batch 23 Loss: 13.746
  Batch: 24
Batch 24 Loss: 14.453
  Batch: 25
Batch 25 Loss: 14.371
  Batch: 26
Batch 26 Loss: 13.404
  Batch: 27
Batch 27 Loss: 14.067
  Batch: 28
Batch 28 Loss: 13.583
  Batch: 29
Batch 29 Loss

KeyboardInterrupt: 

In [ ]:
#Accuracy
cc_vae.eval() 

test_acc = cc_vae.accuracy(test_loader)

print(test_acc)

tensor(0.4944, device='cuda:0')


In [29]:
#Intervention

def intervention(model, x, y):
    post_params = model.encoder(x)
    z = dist.Normal(*post_params).rsample()
    zc, zs = z.split([model.z_classify, model.z_style], 1)
    locs_p_zc, scales_p_zc = model.cond_prior(1 - y)
    zc_new = dist.Normal(locs_p_zc, scales_p_zc).rsample()
    z_new = torch.cat((zc_new,zs),dim=1)
    recons = model.decoder(z_new,x)
    return recons

In [30]:
value2word = {v: k for k, v in word2value.items()}

In [31]:
def decode_recons(recons):
    recons = recons.round()
    r = torch.zeros(recons.shape[1])
    for i in range(recons.shape[1]):
        r[i] = torch.argmax(recons[0][i]) 
    decoded_review = [value2word.get(idx.item(), '<UNK>') for idx in r]
    decoded_text = '.'.join(decoded_review)
    return decoded_text

In [ ]:
k = 0
C = 3
for (i,a ,b) in test_loader :
    while k < C:
        y = b[k].view(1).to(device)
        x = a[k].view(1,seq_len,input_size).to(device)
        recons = intervention(cc_vae, x, y)
        dec = decode_recons(recons)
        print("True review", final_df.iloc[i[k].item()]['review'])
        print("Modified review", dec)
        k += 1

True review ['this', 'is', 'a', 'pretty', 'pointless', 'remake', '.', 'starting', 'with', 'the', 'opening', 'title', 'shots', 'of', 'the', 'original', 'was', 'a', 'real', 'mistake', 'as', 'it', 'reminds', 'the', 'viewer', 'of', 'what', 'a', 'great', 'little', 'period', 'piece', '<UNK>', 'that', 'was', '.', 'the', 'new', 'version', 'that', 'follows', 'is', 'an', 'exercise', 'in', '<UNK>', '<', 'br', '/', '>', '<', 'br', '/', '>', 'brian', '<UNK>', 'plays', 'a', '<UNK>', 'boy', "'", 'photographer', 'who', 'returns', 'to', 'a', '<UNK>', 'desert', 'town', 'populated', 'by', 'a', '<UNK>', 'of', '<UNK>', 'clichéd', 'stock', 'characters', ':', 'the', '<UNK>', 'sucking', '<UNK>', '<UNK>', ',', 'the', '<UNK>', 'old', '<UNK>', '<UNK>', ',', 'the', 'crippled', 'vet', 'and', 'his', 'asian', 'wife', ',', 'etc', '...', '<', 'br', '/', '>', '<', 'br', '/', '>', '<UNK>', "'s", 'character', 'witnesses', 'the', 'crashing', 'of', '<UNK>', "'", 'into', 'a', '<UNK>', 'and', 'shortly', 'after', 'strange', '

M2

In [ ]:
import gc

# Training
m2 = M2(input_size, z_dim, num_classes, hidden_dim, seq_len, vocab_size, batch_size).to(device)

m2.apply(init_weights)

optim = torch.optim.Adam(params=m2.parameters(), lr=0.0001)

for epoch in range(n_epochs):
    m2.train()
    
    epoch_losses_sup = 0.0
    #for i in tqdm(range(num_batch)):
    for batch_idx, (_, xs, ys) in enumerate(train_loader):
        #xs, ys = next(train_loader)
        xs = xs.view(xs.shape[0], seq_len, input_size).to(device)
        ys = ys.to(device)
        loss, llk, cat, kld_norm = m2.sup(xs, ys)
        optim.zero_grad() 
        loss.backward()
        optim.step()

        epoch_losses_sup += loss.detach().item()
        #print(f"Batch {batch_idx} Loss: {loss.detach().item():.3f}")

        gc.collect()
    
    avg_loss = epoch_losses_sup / len(train_loader)
    print(f"[Epoch {epoch+1:03d}] Sup Loss: {avg_loss:.3f}")

Epoch: 0
  Batch: 0
Batch 0 Loss: 14.608
  Batch: 1
Batch 1 Loss: 14.405
  Batch: 2
Batch 2 Loss: 14.172
  Batch: 3
Batch 3 Loss: 13.987
  Batch: 4
Batch 4 Loss: 13.768
  Batch: 5
Batch 5 Loss: 13.582
  Batch: 6
Batch 6 Loss: 13.422
  Batch: 7
Batch 7 Loss: 13.232
  Batch: 8
Batch 8 Loss: 13.048
  Batch: 9
Batch 9 Loss: 12.870
  Batch: 10
Batch 10 Loss: 12.709
  Batch: 11
Batch 11 Loss: 12.556
  Batch: 12
Batch 12 Loss: 12.429
  Batch: 13
Batch 13 Loss: 12.298
  Batch: 14
Batch 14 Loss: 12.142
  Batch: 15
Batch 15 Loss: 12.011
  Batch: 16
Batch 16 Loss: 11.909
  Batch: 17
Batch 17 Loss: 11.807
  Batch: 18
Batch 18 Loss: 11.687
  Batch: 19
Batch 19 Loss: 11.545
  Batch: 20
Batch 20 Loss: 11.454
  Batch: 21
Batch 21 Loss: 11.354
  Batch: 22
Batch 22 Loss: 11.277
  Batch: 23
Batch 23 Loss: 11.133
  Batch: 24
Batch 24 Loss: 11.033
  Batch: 25
Batch 25 Loss: 10.951
  Batch: 26
Batch 26 Loss: 10.840
  Batch: 27
Batch 27 Loss: 10.782
  Batch: 28
Batch 28 Loss: 10.692
  Batch: 29
Batch 29 Loss

In [ ]:
#Accuracy
m2.eval() 

test_acc = m2.accuracy(test_loader)

print(test_acc)

tensor(0.4939)
